In [1]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path
from keras.preprocessing.image import img_to_array

from keras.datasets import mnist
import numpy as np
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path

from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.utils import to_categorical

import tensorflow as tf

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from tensorflow.keras.regularizers import l2

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout

from scipy.ndimage import gaussian_filter


# Data Cleaning & Processing

In [2]:
# importing and resizing image data 
# change path to wherever your data lives on your computer
csv_path = "/Users/vviswanathan/Documents/GitHub/w207-final-project/petfinder-pawpularity-score/train.csv"
image_path = '/Users/vviswanathan/Documents/GitHub/w207-final-project/petfinder-pawpularity-score/train/'
model = ResNet50(weights='imagenet')

def CutenessBuckets(pawpularity_score):
    if pawpularity_score <= 10:
        return 0
    elif pawpularity_score <= 20:
        return 1
    elif pawpularity_score <= 30:
        return 2
    elif pawpularity_score <= 40:
        return 3
    elif pawpularity_score <= 50:
        return 4
    elif pawpularity_score <= 60:
        return 5
    elif pawpularity_score <= 70:
        return 6
    elif pawpularity_score <= 80:
        return 7
    elif pawpularity_score <= 90:
        return 8
    elif pawpularity_score <= 100:
        return 9

def getAdditionalTags(model, imArr):
    tags=[]
    im = np.expand_dims(imArr, axis=0)
    im = preprocess_input(im)
    preds = model.predict(im)
    #print('Predicted:', decode_predictions(preds, top=3)[0])
    return decode_predictions(preds, top=3)[0][0][0], decode_predictions(preds, top=3)[0][1][0], decode_predictions(preds, top=3)[0][2][0] 
    #return tags

def CleanandProcessData(csv_path, image_path, image_resize):

    # creating path to image using the id in the data frame we've created
    df['path'] = image_path + df['Id'] + '.jpg'
    df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((image_resize,image_resize)))
    
    df['cuteness_bin'] = df['Pawpularity'].apply(CutenessBuckets)
    df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

    labels_raw=[]
    labels_bin = []
    imgArr=[]
    tags=[]

    for ind in df.index:
        tags_data=[]
        img = df.img_array[0]
        t1,t2,t3=getAdditionalTags(model, img)
        imgArr.append(img)
        labels_raw.append(df['Pawpularity'][ind])
        labels_bin.append(df['cuteness_bin'][ind])
        
        #append all metadata in the dataframe as tags
        if (df['Subject Focus'][ind]==1):
            tags_data.append('Subject Focus')
        if (df['Eyes'][ind] ==1):
            tags_data.append('Eyes')
        if (df['Face'][ind] ==1):
            tags_data.append('Face')
        if (df['Near'][ind] ==1):
            tags_data.append('Near')
        if (df['Accessory'][ind] ==1):
            tags_data.append('Accessory')
        if (df['Group'][ind] ==1):
            tags_data.append('Group')
        if (df['Collage'][ind] ==1):
            tags_data.append('Collage')
        if (df['Human'][ind] ==1):
            tags_data.append('Human')
        if (df['Occlusion'][ind] ==1):
            tags_data.append('Occlusion')
        if (df['Info'][ind] ==1):
            tags_data.append('Info')
        if (df['Blur'][ind] ==1):
            tags_data.append('Info')
        tags_data.append(t1)
        tags_data.append(t2)
        tags_data.append(t3)
        tags.append(tags_data)
    #print(tags_data)
    labels_raw = np.array(labels_raw)
    labels_bin = np.array(labels_bin)
    tags= np.array(tags,dtype=object)
    X_train=np.array(imgArr, dtype=float)
    print('Training Shape:',X_train.shape)
    print('Labels Shape;', labels_raw.shape)
    return X_train, labels_raw, labels_bin, tags

# imnporting csv with image ids and pawpularity scores
df = pd.read_csv(csv_path)
X_train, labels_raw, labels_bin, tags = CleanandProcessData(csv_path, image_path, 224)
print(df.head)
print(tags[0])

Training Shape: (98, 224, 224, 3)
Labels Shape; (98,)
<bound method NDFrame.head of                                   Id  Subject Focus  Eyes  Face  Near  Action  \
0   0007de18844b0dbbb5e1f607da0606e0              0     1     1     1       0   
1   0009c66b9439883ba2750fb825e1d7db              0     1     1     0       0   
2   0013fd999caf9a3efe1352ca1b0d937e              0     1     1     1       0   
3   0018df346ac9c1d8413cfcc888ca8246              0     1     1     1       0   
4   001dc955e10590d3ca4673f034feeef2              0     0     0     1       0   
..                               ...            ...   ...   ...   ...     ...   
93  028d7652721dfd8643af1c9b21603af0              0     1     1     1       0   
94  029141b506deb62d9c9e30cb52c952c4              0     1     1     1       0   
95  029eb81eec4951bf0919743a0fdc3ebc              0     1     1     1       0   
96  02a5a6417e5eb9694203e348741f76d0              0     1     1     1       0   
97  02b40c9bb4aa551021c92

In [7]:
# Shuffling data and separating into train, test and dev sets
shuffle = np.random.permutation(np.arange(X_train.shape[0]))
X_train, labels_raw, labels_bin = X_train[shuffle], labels_raw[shuffle], labels_bin[shuffle]

print('data shape: ', X_train.shape)
print('pawpularity label shape:', labels_raw.shape)
print('bin label shape:', labels_bin.shape)

# Set some variables to hold test, dev, and training data.
train_data, train_labels_raw, train_labels_bin = X_train[:7921], labels_raw[:7921], labels_bin[:7921]
test_data, test_labels_raw, test_labels_bin = X_train[8916:], labels_raw[8916:], labels_bin[8916:]
dev_data, dev_labels_raw, dev_labels_bin = X_train[7920:8916], labels_raw[7920:8916], labels_bin[7920:8916]

#train_data, train_labels_raw, train_labels_bin, train_tags = X_train[:79], labels_raw[:79], labels_bin[:79], tags[:79]
#test_data, test_labels_raw, test_labels_bin, test_tags = X_train[89:], labels_raw[89:], labels_bin[89:], tags[:89]
#dev_data, dev_labels_raw, dev_labels_bin, dev_tags = X_train[79:89], labels_raw[79:89], labels_bin[79:89], tags[79:89]

# raw labels (ie original pawpularity score)
test_labels_category = to_categorical(test_labels_raw)
train_labels_category = to_categorical(train_labels_raw)
dev_labels_category = to_categorical(dev_labels_raw)

# bucketed labels 
test_labels_bins_category = to_categorical(test_labels_bin)
train_labels_bins_category = to_categorical(train_labels_bin)
dev_labels_bins_category = to_categorical(dev_labels_bin)

# blurred image data (optional use)
blurred_train = gaussian_filter(train_data, sigma=1)
blurred_test = gaussian_filter(test_data, sigma = 1)

data shape:  (98, 224, 224, 3)
pawpularity label shape: (98,)
bin label shape: (98,)


# First CNN Model 

In [4]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 101
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
print("train_data",train_data.shape )
print("train_lables_cat",train_labels_category.shape )
print("test_data",test_data.shape )
print("test_lables_cat",test_labels_category.shape )

model.fit(train_data,train_labels_category, validation_data=(test_data, test_labels_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
train_data (79, 224, 224, 3)
train_lables_cat (79, 101)
test_data (9, 224, 224, 3)
test_lables_cat (9, 101)
Epoch 1/3
3/3 [==============================] - 6s 2s/step - loss: 126788.3516 - accuracy: 0.0127 - val_loss: 28015.2070 - val_accuracy: 0.0000e+00
Epoch 2/3
3/3 [==============================] - 5s 1s/step - loss: 20427.8379 - accuracy: 0.0253 - val_loss: 9792.0137 - val_accuracy: 0.0000e+00
Epoch 3/3
1/1 [==============================] - 0s 107ms/step - loss: 4.6133 - accuracy: 0.1111

Test loss: 4.613306999206543
Test accuracy: 0.1111111119389534


# Same Model but using 10 category buckets for scores

In [5]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 10
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
model.fit(train_data,train_labels_bins_category, validation_data=(test_data, test_labels_bins_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
Epoch 1/3
3/3 [==============================] - 7s 2s/step - loss: 217338.3750 - accuracy: 0.1392 - val_loss: 57055.8203 - val_accuracy: 0.2222
Epoch 2/3
3/3 [==============================] - 5s 2s/step - loss: 47199.1055 - accuracy: 0.1646 - val_loss: 2.3002 - val_accuracy: 0.1111
Epoch 3/3
1/1 [==============================] - 0s 109ms/step - loss: 1146.2106 - accuracy: 0.1111

Test loss: 1146.2105712890625
Test accuracy: 0.1111111119389534


# Second CNN Model - Using raw paw scores

In [6]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)
tags_input = tf.keras.Input(shape=shape)
#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = [inputs, tags_input], outputs = output)

model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")

history = model.fit(
    data_augmentation.flow(train_data, train_tags, train_labels_category,batch_size=32),
    validation_data = (test_data, test_tags, test_labels_category),
    steps_per_epoch = len(train_data) // 32,
    epochs = 3
)
score = model.evaluate(test_data, test_labels_category, verbose=1)
print('Test accuracy:', score[1])

TypeError: flow() got multiple values for argument 'batch_size'

# Second CNN Model - Using bucket paw scores

In [ ]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs = inputs, outputs = output)

model.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

data_augmentation = ImageDataGenerator(
    rotation_range = 15, 
    zoom_range = 0.15,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    shear_range = 0.1,
    horizontal_flip = True, 
    fill_mode = "nearest")
history = model.fit(
    data_augmentation.flow(train_data,train_labels_bins_category,batch_size=32),
    validation_data = (test_data, test_labels_bins_category),
    steps_per_epoch = len(train_data) // 32,
    epochs = 3
)
score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
print('Test accuracy:', score[1])

# Trying other models (Naive Bayes & KNN) - raw paw scores

In [ ]:
# Other model attempts:
# have to reshape the data to input it into a different model
train_size = train_data.shape[0]
train_shape = train_data.shape[1]*train_data.shape[2]*train_data.shape[3]

test_size = test_data.shape[0]
test_shape = test_data.shape[1]*test_data.shape[2]*test_data.shape[3]

train_data_temp = train_data.reshape(train_size,train_shape)
test_data_temp = test_data.reshape(test_size,test_shape)
print('-'*10, ' K Nearest Neighbor ', '-'*10)
for k in [1,2,3,5,7]:
    knnmodel = KNeighborsClassifier(n_neighbors=k)
    knnmodel.fit(train_data_temp, train_labels_raw)
    test_predicted_labels = knnmodel.predict(test_data_temp)
    print("accuracy with k=", k, " : ", metrics.accuracy_score(test_labels_raw, test_predicted_labels))


print('-'*10, 'Naive Bayes ', '-'*10)
naive_bayes_model1 = BernoulliNB()
naive_bayes_model1.fit(train_data_temp, train_labels_raw)
print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels_raw))

In [ ]:
print('-'*10, ' K Nearest Neighbor ', '-'*10)
for k in [1,2,3,5,7]:
    knnmodel = KNeighborsClassifier(n_neighbors=k)
    knnmodel.fit(train_data_temp, train_labels_bin)
    test_predicted_labels = knnmodel.predict(test_data_temp)
    print("accuracy with k=", k, " : ", metrics.accuracy_score(test_labels_bin, test_predicted_labels))

print('-'*10, 'Naive Bayes ', '-'*10)
naive_bayes_model1 = BernoulliNB()
naive_bayes_model1.fit(train_data_temp, train_labels_bin)
print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels_bin))

In [ ]:

def CutenessBuckets_2(pawpularity_score):
    if pawpularity_score <= 50:
        return 0
    elif pawpularity_score <= 100:
        return 1

    
def CutenessBuckets_3(pawpularity_score):
    if pawpularity_score <= 33:
        return 0
    elif pawpularity_score <= 66:
        return 1
    elif pawpularity_score <= 100:
        return 2

def CutenessBuckets_4(pawpularity_score):
    if pawpularity_score <= 25:
        return 0
    elif pawpularity_score <= 50:
        return 1
    elif pawpularity_score <= 75:
        return 2
    elif pawpularity_score <= 100:
        return 3
df['cuteness_bin10'] = df['Pawpularity'].apply(CutenessBuckets)
df['cuteness_bin2'] = df['Pawpularity'].apply(CutenessBuckets_2)
df['cuteness_bin3'] = df['Pawpularity'].apply(CutenessBuckets_3)
df['cuteness_bin4'] = df['Pawpularity'].apply(CutenessBuckets_4)

labels_raw10 = np.array(df['cuteness_bin10'])
labels_raw2 = np.array(df['cuteness_bin2'])
labels_raw3 = np.array(df['cuteness_bin3'])
labels_raw4 = np.array(df['cuteness_bin4'])

train_labels_raw10 =  labels_raw10[:7921]
test_labels_raw10 = labels_raw10[8916:]

train_labels_raw2 =  labels_raw2[:7921]
test_labels_raw2 = labels_raw2[8916:]

train_labels_raw3 =  labels_raw3[:7921]
test_labels_raw3 = labels_raw3[8916:]

train_labels_raw4 =  labels_raw4[:7921]
test_labels_raw4 = labels_raw4[8916:]

In [ ]:
train_labels_list = [train_labels_raw10, train_labels_raw2, train_labels_raw3, train_labels_raw4]
test_labels_list = [test_labels_raw10, test_labels_raw2, test_labels_raw3, test_labels_raw4]
bucket_list = [10,2,3,4]
for k in [1,2,3,5,7]:
    for i in range(len(train_labels_list)):
        knnmodel = KNeighborsClassifier(n_neighbors=k)
        knnmodel.fit(train_data_temp, train_labels_list[i])
        test_predicted_labels = knnmodel.predict(test_data_temp)
        print("bucket size = :" ,bucket_list[i])
        print("accuracy with k=", k, " : ", metrics.accuracy_score(test_labels_list[i], test_predicted_labels))

In [ ]:
for i in range(len(train_labels_list)):
    naive_bayes_model1 = BernoulliNB()
    naive_bayes_model1.fit(train_data_temp, train_labels_list[i])
    print("bucket size = :" ,bucket_list[i])
    print ('Bernoulli Model accuracy: %3.4f' %naive_bayes_model1.score(test_data_temp, test_labels_list[i]))

In [ ]:
df['path'] = image_path + df['Id'] + '.jpg'
df = df.head(500)
df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((224,224)))

df['cuteness_bin'] = df['Pawpularity'].apply(CutenessBuckets)
df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

labels_raw=[]
labels_bin = []
imgArr=[]

for ind in df.index:
    img = df.img_array[0]
    imgArr.append(img)
    labels_raw.append(df['Pawpularity'][ind])
    labels_bin.append(df['cuteness_bin'][ind])

labels_raw = np.array(labels_raw)
labels_bin = np.array(labels_bin)
X_train=np.array(imgArr, dtype=float)


In [ ]:
#model = ResNet50(weights='imagenet')


def getAdditionalTags(model, imArr):
    tags=[]
    im = np.expand_dims(imArr, axis=0)
    im = preprocess_input(im)
    preds = model.predict(im)
    #print('Predicted:', decode_predictions(preds, top=3)[0])
    tags.append(decode_predictions(preds, top=3)[0][0][0])
    tags.append(decode_predictions(preds, top=3)[0][1][0])   
    tags.append(decode_predictions(preds, top=3)[0][2][0]) 
    return tags
    
def augmentExif(img,df):
    exifData = img._getexif()
    if exifData is None:
        return
    else:
        print (type(exifData))
        for tagId in exifData:
            # get the tag name, instead of human unreadable tag id
            tag = TAGS.get(tagId, tagId)
            data = exifdata.get(tagId)
            # decode bytes 
            if isinstance(data, bytes):
                data = data.decode()
                print(f"{tag:25}: {data}")

In [ ]:
getAdditionalTags(model,imgArr[0])